In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = np.load('../data/raw/fashion_train.npy')
Y = data[:, -1]
X = data[:, :-1]
unique_labels = np.unique(Y)
unique_labels

### PCA Analysis and reconstruction

TODO:
* Create a heatmap of most important pixels according to PCA, using cosine correlations. Like, one image.
* Create a template by averaging images within each class. Then, do reconstruction. Then for each image obtain only one value using pearson correlation.
* Other way: do reconstruction of each image and then do the templates. Then, for each image obtain only one values using pearson correlation.
* See the correlation between these two :)
* reconstruct several images, present them according to a few PCA thresholds. (i propose 1, 10, 71, 500, 784)
* do the 3d Biplot (even though it's useless xd)

Heatmap of most important pixels according to PCA

In [3]:
from sklearn.decomposition import PCA
pca = PCA(n_components=71)
pca.fit(X)
def pca_inverse_transform(n):
    return pca.transform(X)[:, :n] @ pca.components_[:n] + pca.mean_


In [ ]:
pca = PCA(n_components=None)
pca.fit(X)

evr = np.hstack([0, pca.explained_variance_ratio_])
c_evr = np.cumsum(evr)

plt.figure()
plt.plot(c_evr)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance (%)') #for each component
# vertical line at 71th component
plt.axvline(x=71, color='r', linestyle='--', label='71 components', linewidth=0.75)
plt.axhline(y=c_evr[71], color='g', linestyle='--', label=f'{c_evr[71]*100:.2f}%', linewidth=0.75)
plt.axvline(x=0, color='k', linewidth=0.5) # make it thin
plt.axhline(y=1, color='k', linewidth=0.5)
plt.legend()
plt.show()


plt.figure()
plt.plot(evr[:80])
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.axvline(x=71, color='r', linestyle='--', label='71 components', linewidth=0.75)
plt.axhline(y=evr[71], color='g', linestyle='--', label=f'{evr[71]*100:.2f}%')
plt.legend()
plt.show()


In [ ]:
r = pca_inverse_transform(71)

fig,ax = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))
fig.suptitle('Ground truth vs Reconstructed images for 71 components')


for ax_i, img_i in enumerate([500, 600,748, 900, 71]):
    img = X[img_i, :].reshape((28,28))
    r_img = r[img_i, :].reshape((28,28))
    ax[0, ax_i].imshow(img, cmap='gray')
    ax[1, ax_i].imshow(r_img, cmap='gray')
plt.show()

# as we see, for 71 parameters there is almost no change! that's so great :)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Load data
data = np.load('../data/raw/fashion_train.npy')
Y = data[:, -1]
X = data[:, :-1]

# Perform PCA

# Compute total explained variance by pixel
def total_explained_variance_by_pixel(pca: PCA, selection: range=None):
    components = pca.components_
    evr = pca.explained_variance_ratio_
    if selection is not None:
        components = components[selection.start:selection.stop:selection.step]
        evr = evr[selection.start:selection.stop:selection.step]
        
    ev = np.abs(components.T) @ evr
    
    print(evr.sum())
    ttl_ev = evr.sum() * ( ev / ev.sum() )
    return ttl_ev.reshape((28,28))

ttl_ev = total_explained_variance_by_pixel(pca, range(0, 71))

# Visualize
plt.imshow(ttl_ev, cmap='hot')
plt.title('A pixel PCA importance based on sum of weighted loadings of 71 PCs')
plt.colorbar()
plt.show()



### Importance based on first four components

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 5))
fig.suptitle('Pixel PCA Importance Based on Weighted Loadings of First Four PCs')

for i in range(4):
    ttl_ev = total_explained_variance_by_pixel(pca, range(i, i+1))
    ax[i].imshow(ttl_ev, cmap='hot')
    ax[i].set_title(f'{i}th PC, Expl. variance: {pca.explained_variance_ratio_[i]*100:.2f}%')
    ratio = pca.explained_variance_ratio_[i]
    # colorbar
    fig.colorbar(ax[i].imshow(ttl_ev, cmap='hot'), ax=ax[i])

plt.show()

In [32]:
# Type of clothing T-shirt/top Trouser Pullover Dress Shirt
# Label 0 1 2 3 4

# mapping for labels:
label_map = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Shirt'
}

mapped_labels = np.vectorize(lambda x: label_map[x])(Y)

In [ ]:
### 3d plot of first 3 components

### create three subplots, plotting datapoints on axes determined by pairs of first three components
from itertools import combinations

pca_3 = PCA(n_components=3)
X_pca = pca_3.fit_transform(X)

# pick random 1000 samples
idx = np.random.choice(X.shape[0], 500, replace=False)
X_pca = X_pca[idx]
labels = Y[idx]


fig, ax = plt.subplots(1, 3, figsize=(15,5))
fig.suptitle('First Three Principal Components')

for i, pair_idx in enumerate(combinations(range(3), 2)):
    o = ax[i].scatter(X_pca[:, pair_idx[0]], X_pca[:, pair_idx[1]], c=labels, cmap='tab10', alpha=0.5)
    ax[i].set_xlabel(f'PC{pair_idx[0]+1}')
    ax[i].set_ylabel(f'PC{pair_idx[1]+1}')
plt.legend(*o.legend_elements(), title='Classes')
plt.show()


In [12]:
%matplotlib widget

In [ ]:
o.legend_elements()

In [ ]:
# Ensure interactive plotting
plt.ion()


In [ ]:
import matplotlib.pyplot as plt

# Ensure interactive plotting
plt.ion()

# Create rotatable 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
o = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=labels, cmap="tab10", alpha=0.5)
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")

plt.legend(*o.legend_elements(), title="Classes")

ax.view_init(10, 40)


# Show the plot
plt.show()



I think it confirms that methods focused on contour based methods might be the best, because most helpful information is around.

Secondly, there is a clear shape of trousers in the middle. I don't think it's very significant - though it means that most probably trousers are 'within' other clothes images and could be detected based on smallest area, and perhaps this white vertical line in the middle.

### Some basic metrics for each of the classes - variance, mean, std of pixels within class.

In [ ]:
# Show the distribution of classes
unique, counts = np.unique(Y, return_counts=True)
plt.bar(unique, counts)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.title('Distribution of Classes')
plt.show()


In [ ]:

# Compute the metrics for 'average' image within each class
class_metrics = {}
for label in unique:
    class_images = X[Y == label]
    average_image = np.mean(class_images, axis=0)
    variance = np.var(average_image)
    mean = np.mean(average_image)
    std = np.std(average_image)
    class_metrics[label] = {'mean': mean, 'variance': variance, 'std': std}

# format as a nice table for display
print('Class\tMean\tVariance\tStandard Deviation')
for label, metrics in class_metrics.items():
    print('{} \t{:.2f} \t{:.2f} \t{:.2f}'.format(label, metrics['mean'], metrics['variance'], metrics['std']))



In [ ]:
[X_mean[Y == label] for label in unique]

In [ ]:
# now, summarize each image as just a mean value
# then, do the boxplot with regards to classes
X_mean = np.mean(X, axis=1)
plt.boxplot([X_mean[Y == label] for label in unique])
plt.xlabel('Class')
plt.ylabel('Mean Pixel Value')
plt.title('Distribution of Mean Pixel Value by Class')
plt.show()

In [ ]:
# now, show the distributions of classes' means of pixel values. These will follow a normal distribution as these are means of pixel values
for label in unique_labels:
    plt.hist(X_mean[Y == label], bins=20, alpha=0.5, label='Class {}'.format(label))
plt.xlabel('Mean Pixel Value')
plt.show()


# these follow a normal distribution, as expected
# we can do t-tests to see if the means are significantly different.
# under the null hypothesis, the means are the same

from scipy.stats import ttest_ind
for i in range(len(unique_labels)):
    for j in range(i+1, len(unique_labels)):
        class1 = X_mean[Y == i]
        class2 = X_mean[Y == j]
        t, p = ttest_ind(class1, class2)
        # if p < 0.05, we reject the null hypothesis that the means are the same
        if p < 0.05:
            print('Classes {} and {} have significantly different mean pixel values'.format(i, j))
        else:
            print('Classes {} and {} do not have significantly different mean pixel values'.format(i, j))

# now, show the distribution of classes' variance of pixel values. These will follow a chi-squared distribution as these are variances of pixel values
X_var = np.var(X, axis=1)
for label in unique_labels:
    plt.hist(X_var[Y == label], bins=20, alpha=0.5, label='Class {}'.format(label))
plt.xlabel('Variance of Pixel Value')
plt.show()

# these follow a chi-squared distribution, as expected
# we can do F-tests to see if the variances are significantly different.
# under the null hypothesis, the variances are the same

from scipy.stats import f_oneway
for i in range(len(unique_labels)):
    for j in range(i+1, len(unique_labels)):
        class1 = X_var[Y == i]
        class2 = X_var[Y == j]
        f, p = f_oneway(class1, class2)
        # if p < 0.05, we reject the null hypothesis that the variances are the same
        if p < 0.05:
            print('Classes {} and {} have significantly different variances of pixel values'.format(i, j))
        else:
            print('Classes {} and {} do not have significantly different variances of pixel values'.format(i, j))

As we have seen above - the mean and the variances follow the known distributions. Would it be possible to create a distribution which is in one direction normal whereas in second chi-squared? I don't think so. But maybe? The simplest way would be to just treat them as independent (mean from variance) but it's totally wrong. I believe there must be ready models to do so!

# Computing the templates
So, the ideas are: 
1. do the PCA reduction and then get the 'template' / 'heatmap'
2. get the 'template' / 'heatmap' and then do the PCA reduction

For now my naive assumption is, that the 'heatmap' is just mean, which can be later multiplied to get the values. With the pca

Start:

In [ ]:
# get the 'heatmap' of the pixel values for each class
heatmaps = []
_, ax = plt.subplots(ncols=len(unique_labels))
for i, label in enumerate(unique_labels):
    heatmap = np.mean(X[Y == label], axis=0)
    heatmaps.append(heatmap)
    ax[i].imshow(heatmap.reshape((28,28)), cmap='gray')
    plt.title('Class {}'.format(label))
plt.show()
heatmaps = np.array(heatmaps)



# here I don't do entrywise multiplication first and then sum, I directly do the dot product. That won't be the case of reverse operations
heatmap_scores_after_pca = reduced_original_coordinates @ heatmaps.T


class_heatmap_scores_after_pca = []
for label1 in unique_labels:
    for label2 in [label1, *unique_labels[list(unique_labels).index(label1)+1:]]:
        class_heatmap_scores_after_pca.append(heatmap_scores_after_pca[Y == label1][:,label2])
plt.figure()
plt.boxplot(class_heatmap_scores_after_pca)
plt.title('Distribution of Heatmap Scores by Class')
plt.show()

From looking at the pictures: I think it could be a great feature, seriously!

From looking at the boxplots I think it could be a valuable tool for certin classes, but not for the first one apparently.

In [199]:

# second idea: do template matching with the heatmaps (so do the entrywise mutliplication first) and then do PCA and then do the dot product
# so first 'fit' the templates for each class

fitted_templates = X[np.newaxis, :, :] * heatmaps[:, np.newaxis, :]
fitted_templates=fitted_templates.reshape((5*10000, 784))

# pca on the fitted templates

heatmap_pca = PCA(n_components=71)
heatmap_pca_coordinates = pca.fit_transform(fitted_templates)
heatmap_reduced_original_coordinates = pca.inverse_transform(heatmap_pca_coordinates)

heatmap_reduced_original_coordinates = heatmap_reduced_original_coordinates.reshape((5, 10000, 784))
# now, take the sum of each heatmap (5) for each image (10000)
heatmap_scores_before_pca = np.sum(heatmap_reduced_original_coordinates, axis=2).T


In [ ]:
# now, we need to get boxplots again.
# so basically, the array is 5*10000 x 784
# it's composed by stacking 5 heatmaps of 10000 images, labels are in Y.
# I need to have 15 buckets, each against each other incl. itself. 
# so now it's about disassembling heatmap_reduced_original_coordinates

class_heatmap_scores_before_pca = []
for label1 in unique_labels:
    for label2 in [label1, *unique_labels[list(unique_labels).index(label1)+1:]]:
        class_heatmap_scores_before_pca.append(heatmap_scores_before_pca[Y == label1][:,label2])
plt.figure()
plt.boxplot(class_heatmap_scores_before_pca)
plt.title('Distribution of Heatmap Scores by Class')
plt.show()

In [ ]:
plt.plot([np.mean(x) for x in class_heatmap_scores_before_pca], [np.mean(x) for x in class_heatmap_scores_after_pca])

There is perfect correlation which makes obviously sense. So, there is no reason to do differently.